# Lesson 01: Electron Lifetime

The goal of this lesson is to illustrate the electron lifetime calibration and correction as performed in LXe-TPCs.

## Learning Objectives
Physics Topics:
- Energy deposition in LXe
- Electron transport in LXe
- Electroluminesence gain

Mathematics Topics:
- Expoential decays
- Least-squares regression

Programming Topics:
- Working with arrays
- Visualizing data